In [2]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
from utils import processLasairData

project_root = Path.cwd().parent
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

ztf_data = project_root / "ztf_cleansed.csv"

ztf_df = pd.read_csv(ztf_data)

print(ztf_df.head())

          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF19aamhqej  SN2019bvt  18:28:48.43  +50:22:23.4  58572.51        g   

   peakmag  peakabs  duration    rise    fade   type  redshift          b  \
0  15.4052    -19.1     21.52  10.257  11.263  SN Ia    0.0169  24.026989   

     A_V  
0  0.158  


In [ ]:
from typing import Any

lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    try:
        result = client.object(ztf_id)
    except Exception as e:
        print(f"\nLasair API error for {ztf_id}: {e}")
        continue

    if not result or 'candidates' not in result:
        print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
        continue
    if "error" in result:
        print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
        continue

    candidates = result['candidates']
    if not candidates:
        print(f"\nNo light curve data for {ztf_id}")
        continue

    output_rows = []
    for cand in candidates:
        # fid: filter ID (1=g, 2=r)
        fid = cand.get("fid")
        fid_to_letter = {1: "g", 2: "r", 3: "i"}
        filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None

        out_row = {
            'ztf_id': ztf_id,
            'iauid': iauid,
            'MJD': cand.get('mjd', None),
            'filter': filter_letter,
            'unforced_mag': cand.get('magpsf', None),
            'unforced_mag_error': cand.get('sigmapsf', None),
            'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
            'forced_ujy': cand.get('forcediffimflux', None),
            'forced_ujy_error': cand.get('forcediffimfluxunc', None),
        }
        output_rows.append(out_row)

    # Save per-object light curve CSV in run_folder
    df_obj = pd.DataFrame(output_rows)
    out_path = run_folder / f"{ztf_id}_lightcurve.csv"
    df_obj.to_csv(out_path, index=False)
    print(f" Saved {len(output_rows)} rows to {out_path.name}.", end=' ')
    try:
        processLasairData(df_obj, out_path)
        num_success += 1
        print(f"Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f"Error running processLasairData: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/1: ZTF19aamhqej... Saved 126 rows to ZTF19aamhqej_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test1/ZTF19aamhqej_lightcurve.csv (76 rows)
Done (1/1)

All done! 1 of 1 objects processed successfully.
